# Nosso desafio

- Para cada aeroporto trazer as top10 **companhias aéreas com maior atuação no ano** com as seguintes informações:
    - Razão social da Companhia Aérea
    - ICAO do Aeroporto
    - Quantidade de Rotas à partir daquele aeroporto
    - Quantidade de Rotas com destino àquele aeroporto
    - Quantidade total de pousos e decolagens naquele aeroporto

# Solving

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [36]:
df_vra = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)

df_vra.show(1, vertical=True)

df_aircia  = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/air_cia/")
)

df_aircia.show(1, vertical=True)

-RECORD 0----------------------------------
 icao_empresa_aerea     | AAL              
 numero_voo             | 904              
 codigo_di              | 0                
 codigo_tipo_linha      | I                
 icao_aerodromo_origem  | SBGL             
 icao_aerodromo_destino | KMIA             
 partida_prevista       | 01/06/2022 23:00 
 partida_real           | 01/06/2022 22:57 
 chegada_prevista       | 02/06/2022 07:45 
 chegada_real           | 02/06/2022 07:17 
 situacao_voo           | REALIZADO        
 codigo_justificativa   | null             
only showing top 1 row

-RECORD 0----------------------------------------
 razao_social             | ABSA - AEROLINHAS... 
 icao_iata                | LTG M3               
 cnpj                     | 00074635000133       
 atividades_aereas        | TRANSPORTE AÉREO ... 
 endereco_sede            | AEROPORTO INTERNA... 
 telefone                 | 1155828055           
 e-mail                   | gar@tam.com.br       
 dec

In [4]:
df_aircia.createOrReplaceTempView("cia_aerea")
df_vra.createOrReplaceTempView("vra")

In [5]:
spark.sql("select * from vra limit 10").show()

+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+
|icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|partida_prevista|    partida_real|chegada_prevista|    chegada_real|situacao_voo|codigo_justificativa|
+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+
|               AAL|       904|        0|                I|                 SBGL|                  KMIA|01/06/2022 23:00|01/06/2022 22:57|02/06/2022 07:45|02/06/2022 07:17|   REALIZADO|                null|
|               AAL|       905|        0|                I|                 KMIA|                  SBGL|01/06/2022 21:00|01/06/2022 21:55|02/06/2022 05:10|02/06/2022 06:07|

In [6]:
spark.sql("select * from cia_aerea limit 10").show()

+--------------------+---------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|        razao_social|icao_iata|          cnpj|   atividades_aereas|       endereco_sede|            telefone|              e-mail| decisao_operacional|data_decisao_operacional|validade_operacional|icao|iata|
+--------------------+---------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|ABSA - AEROLINHAS...|   LTG M3|00074635000133|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|          1155828055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025| LTG|  M3|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27315694000102|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|          4331764030|operacoes@aerosul...|      DECISÃO Nº 282|            

In [10]:
spark.sql("""
SELECT
     YEAR(to_date(vra.partida_real,'dd/MM/yyyy HH:mm')) AS Ano,
     cia.razao_social AS Razao_social_CIA,
     vra.icao_empresa_aerea AS ICAO_do_Aeroporto,
     COUNT(vra.icao_aerodromo_origem) AS Quantidade_Origem,
     COUNT(vra.icao_aerodromo_destino) AS Quantidade_Destino,
     (COUNT(vra.icao_aerodromo_origem) + COUNT(vra.icao_aerodromo_destino)) AS Quantidade_Total
FROM
    vra
        LEFT JOIN cia_aerea cia ON vra.icao_empresa_aerea = cia.icao
WHERE
   vra.situacao_voo = 'REALIZADO'
GROUP BY 
    YEAR(to_date(vra.partida_real,'dd/MM/yyyy HH:mm')),
    cia.razao_social,
    vra.icao_empresa_aerea
ORDER BY
    YEAR(to_date(vra.partida_real,'dd/MM/yyyy HH:mm')), 
    (COUNT(vra.icao_aerodromo_origem) + COUNT(vra.icao_aerodromo_destino))
DESC
""").show()

+----+--------------------+-----------------+-----------------+------------------+----------------+
| Ano|    Razao_social_CIA|ICAO_do_Aeroporto|Quantidade_Origem|Quantidade_Destino|Quantidade_Total|
+----+--------------------+-----------------+-----------------+------------------+----------------+
|null|AZUL LINHAS AÉREA...|              AZU|           293534|            286809|          580343|
|null|TAM LINHAS AÉREAS...|              TAM|           250372|            245447|          495819|
|null|GOL LINHAS AÉREAS...|              GLO|           230594|            222394|          452988|
|null|AZUL CONECTA LTDA...|              ACN|            18904|             16711|           35615|
|null|PASSAREDO TRANSPO...|              PTB|            16313|             15076|           31389|
|null|SIDERAL LINHAS AÉ...|              SID|            15428|             14380|           29808|
|null|                null|              TAP|             7803|              7679|           15482|


In [12]:
spark.sql("""
with vra_clean as (
select
    YEAR(to_date(partida_prevista,'dd/MM/yyyy HH:mm')) AS Ano,
    *
from 
    vra
where
    vra.situacao_voo = ('REALIZADO')
)
select count(*)
from vra_clean
where ano in ('2022', '2021', '2023', '2020')
""").show()

+--------+
|count(1)|
+--------+
| 2473226|
+--------+



In [14]:
spark.sql("select count(*) from vra where situacao_voo = ('REALIZADO')").show()

+--------+
|count(1)|
+--------+
| 2494954|
+--------+



In [15]:
2494954 - 2473226 

21728

In [25]:
spark.sql("""
with vra_clean as (
select
    YEAR(to_date(partida_prevista,'dd/MM/yyyy HH:mm')) AS Ano,
    *
from 
    vra
where
    vra.situacao_voo = ('REALIZADO')
    and YEAR(to_date(partida_prevista,'dd/MM/yyyy HH:mm')) in ('2022', '2021', '2023', '2020')
),
count_origem as (
select
    vra.Ano,
    cia.razao_social AS Razao_social_CIA,
    vra.icao_empresa_aerea AS ICAO_do_Aeroporto,
    vra.icao_aerodromo_origem,
    COUNT(*) AS Quantidade_Origem,
    rank() OVER (PARTITION BY vra.Ano ORDER BY COUNT(*) DESC) AS rank
from 
    vra_clean as vra LEFT JOIN cia_aerea cia ON vra.icao_empresa_aerea = cia.icao
group by
    vra.Ano,
    cia.razao_social,
    vra.icao_empresa_aerea,
    vra.icao_aerodromo_origem
order by 
    vra.Ano,
    Quantidade_Origem
DESC
),
count_destino as (
select
    vra.Ano,
    cia.razao_social AS Razao_social_CIA,
    vra.icao_empresa_aerea AS ICAO_do_Aeroporto,
    vra.icao_aerodromo_destino,
    COUNT(*) AS Quantidade_Destino,
    rank() OVER (PARTITION BY vra.Ano ORDER BY COUNT(*) DESC) AS rank
from 
    vra_clean as vra LEFT JOIN cia_aerea cia ON vra.icao_empresa_aerea = cia.icao
group by
    vra.Ano,
    cia.razao_social,
    vra.icao_empresa_aerea,
    vra.icao_aerodromo_destino
order by 
    vra.Ano,
    Quantidade_Destino
DESC
)
select 
    o.Ano,
    o.Razao_social_CIA,
    o.ICAO_do_Aeroporto,
    o.icao_aerodromo_origem,
    d.icao_aerodromo_destino,
    o.Quantidade_Origem,
    d.Quantidade_Destino,
    (o.Quantidade_Origem + d.Quantidade_Destino) as Quantidade_Total
from
    count_origem o inner join count_destino d 
    on o.ICAO_do_Aeroporto = d.ICAO_do_Aeroporto
    and o.icao_aerodromo_origem = d.icao_aerodromo_destino
    and o.rank <=10 and d.rank <=10
order by
    o.Ano,
    (o.Quantidade_Origem + d.Quantidade_Destino)
DESC
""").show(40)

+----+--------------------+-----------------+---------------------+----------------------+-----------------+------------------+----------------+
| Ano|    Razao_social_CIA|ICAO_do_Aeroporto|icao_aerodromo_origem|icao_aerodromo_destino|Quantidade_Origem|Quantidade_Destino|Quantidade_Total|
+----+--------------------+-----------------+---------------------+----------------------+-----------------+------------------+----------------+
|2020|AZUL LINHAS AÉREA...|              AZU|                 SBKP|                  SBKP|            28930|             44550|           73480|
|2020|AZUL LINHAS AÉREA...|              AZU|                 SBKP|                  SBKP|            28930|             39523|           68453|
|2020|TAM LINHAS AÉREAS...|              TAM|                 SBGR|                  SBGR|            28941|             37158|           66099|
|2020|TAM LINHAS AÉREAS...|              TAM|                 SBGR|                  SBGR|            28941|             34801|   

In [29]:
from pyspark.sql.window import Window

In [72]:
w1 = Window.partitionBy("Ano").orderBy(F.desc("Quantidade_Destino"))
w2 = Window.partitionBy("Ano").orderBy(F.desc("Quantidade_Origem"))

In [73]:
df_vra_clean = (
    df_vra
    .withColumn("Ano", F.year(F.to_date(F.col("partida_prevista"), 'dd/MM/yyyy HH:mm')))
    .filter(F.col("Ano").isin(['2022', '2021', '2023', '2020']))
)

count_destino = (
    df_vra_clean
    .join(df_aircia,df_vra_clean.icao_empresa_aerea == df_aircia.icao ,"left")
    .withColumn("icao_aerodromo_destino", F.regexp_extract(F.col("icao_aerodromo_destino"), "([a-zA-Z]{4})", 1))
    .groupBy(
        "Ano",
        "razao_social",
        "icao_empresa_aerea",
        "icao_aerodromo_destino"
    )
    .agg(
        F.count(F.col("icao_aerodromo_destino")).alias("Quantidade_Destino")
    )
    .orderBy("Quantidade_Destino", Ascending=False)
)

count_destino_final = (
    count_destino
    .withColumn("rank", F.rank().over(w1))
)

count_origem = (
    df_vra_clean
    .join(df_aircia,df_vra_clean.icao_empresa_aerea == df_aircia.icao ,"left")
    .groupBy(
        "Ano",
        "razao_social",
        "icao_empresa_aerea",
        "icao_aerodromo_origem"
    )
    .agg(
        F.count("icao_aerodromo_origem").alias("Quantidade_Origem")
    )
    .withColumn("rank", F.rank().over(w2))
)

In [38]:
df_vra_clean.show()

+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+----+
|icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|partida_prevista|    partida_real|chegada_prevista|    chegada_real|situacao_voo|codigo_justificativa| Ano|
+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+----+
|               AAL|       904|        0|                I|                 SBGL|                  KMIA|01/06/2022 23:00|01/06/2022 22:57|02/06/2022 07:45|02/06/2022 07:17|   REALIZADO|                null|2022|
|               AAL|       905|        0|                I|                 KMIA|                  SBGL|01/06/2022 21:00|01/06/2022 21:55|02/06/2022 05:

In [70]:
count_destino_final.show(40)

+----+--------------------+------------------+----------------------+------------------+----+
| Ano|        razao_social|icao_empresa_aerea|icao_aerodromo_destino|Quantidade_Destino|rank|
+----+--------------------+------------------+----------------------+------------------+----+
|2020|TAM LINHAS AÉREAS...|               TAM|                  SBGR|             24545|   1|
|2020|AZUL LINHAS AÉREA...|               AZU|                  SBKP|             24534|   2|
|2020|AZUL LINHAS AÉREA...|               AZU|                      |             22247|   3|
|2020|GOL LINHAS AÉREAS...|               GLO|                  SBGR|             20073|   4|
|2020|GOL LINHAS AÉREAS...|               GLO|                      |             19199|   5|
|2020|TAM LINHAS AÉREAS...|               TAM|                      |             16091|   6|
|2020|AZUL LINHAS AÉREA...|               AZU|                  SBCF|             12845|   7|
|2020|TAM LINHAS AÉREAS...|               TAM|              

In [74]:
count_origem.show(40)

+----+--------------------+------------------+---------------------+-----------------+----+
| Ano|        razao_social|icao_empresa_aerea|icao_aerodromo_origem|Quantidade_Origem|rank|
+----+--------------------+------------------+---------------------+-----------------+----+
|2020|AZUL LINHAS AÉREA...|               AZU|                 SBKP|            29584|   1|
|2020|TAM LINHAS AÉREAS...|               TAM|                 SBGR|            29278|   2|
|2020|GOL LINHAS AÉREAS...|               GLO|                 SBGR|            24302|   3|
|2020|AZUL LINHAS AÉREA...|               AZU|                 SBCF|            14949|   4|
|2020|TAM LINHAS AÉREAS...|               TAM|                 SBBR|            13273|   5|
|2020|TAM LINHAS AÉREAS...|               TAM|                 SBSP|            12858|   6|
|2020|AZUL LINHAS AÉREA...|               AZU|                 SBRF|            12660|   7|
|2020|GOL LINHAS AÉREAS...|               GLO|                 SBSP|            